# Corr DMRs

In [1]:
import pandas as pd
import numpy as np
import anndata

In [2]:
motif_meta_with_cluster = pd.read_csv(
    '/home/hanliu/project/mouse_rostral_brain/study/MotifClustering/JASPAR2020_CORE_vertebrates_non-redundant.mouse_genes.with_motif_group.199.csv',
    index_col='motif_uid'
)
tf_class = pd.read_csv('/home/hanliu/ref/TFGene/TFClass/TFClass.with_mouse_gene_id.csv')
tf_class.head()
level = 'Family'
gene_to_tf_class = {}
for _, row in tf_class.iterrows():
    if isinstance(row['EnsemblID'], float):
        continue
    for g in row['EnsemblID'].split(','):
        gene_to_tf_class[g] = row[level]
motif_meta_with_cluster['motif_class'] = motif_meta_with_cluster['gene_ids'].map(
    lambda i: gene_to_tf_class[i.split('.')[0]] if i.split('.')[0] in gene_to_tf_class else '')

level = 'SubFamily'
gene_to_tf_class = {}
for _, row in tf_class.iterrows():
    if isinstance(row['EnsemblID'], float):
        continue
    for g in row['EnsemblID'].split(','):
        gene_to_tf_class[g] = row[level]
motif_meta_with_cluster['motif_class_sub'] = motif_meta_with_cluster['gene_ids'].map(
    lambda i: gene_to_tf_class[i.split('.')[0]] if i.split('.')[0] in gene_to_tf_class else '')

In [4]:
seleted_motifs = motif_meta_with_cluster[motif_meta_with_cluster['motif_class'] == 'Tal-related'].index

## Motif scan

In [5]:
dmr_motif_scan = anndata.read_h5ad(
    '/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial/MotifScan.h5ad'
)
dmr_motif_scan = dmr_motif_scan[:, seleted_motifs].copy()
dmr_motif_scan.X = dmr_motif_scan.X.todense()

## Filter score

In [10]:
# filter DMR the same way as motif enrichment
mask_quantile_to99 = 0.9

# mask small motif scores
motif_cutoff = pd.Series(np.apply_along_axis(lambda i: np.quantile(i[i>0], 0.99) * mask_quantile_to99 if (i[i>0].size > 0) else 0, 
                                             0, dmr_motif_scan.X),
                         index=dmr_motif_scan.var_names)
# only keep value larger than the cutoff for each motif
dmr_motif_scan.X = np.multiply(dmr_motif_scan.X, (dmr_motif_scan.X > motif_cutoff.values[None, :]))

In [23]:
sig_dmrs = dmr_motif_scan.obs_names[(dmr_motif_scan.X.sum(axis=1) > 0).A1]

In [25]:
corr_adata = anndata.read_h5ad(
    '/home/hanliu/project/mouse_rostral_brain/study/ITSpatial/gene_dmr_corr/Corr/chr1.h5ad'
)

In [30]:
corr_adata[corr_adata.obs_names.isin(sig_dmrs), :]

<40347x3359 sparse matrix of type '<class 'numpy.float32'>'
	with 105450 stored elements in Compressed Sparse Row format>